In [34]:
import os

import pandas as pd
import gspread_dataframe as gs
import pyperclip as clip # copiar o texto clipboard
from time import sleep
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup as soup

import random




# Change the directory to reflect the main repository
os.chdir("C:/Users/rsilva/gdrive/projects/programming/automation")

#import sistemas
import sei
import functions
import page
from page import *
import helpers

% reload_ext autoreload
% autoreload 2

In [101]:
TAGS = ["@nome_interessado_maiusculas@", "@endereco_destinatario@", "@bairro_destinatario@", "@cep_destinatario@", "@cidade_destinatario@", "@validade@"]
COLUNAS = ["CPF","Nome", "FISTEL", "Validade", "Filtro", "End_Sede", "End_Corresp.",
           "CPF_RF", "Nome_RF", "Mãe", "Data de Nascimento", "Sexo", "Ano do Óbito", 
           "Nacionalidade", "Título de Eleitor", "Situação Cadastral", "Fone",
           "Logradouro", "Número", "Complemento", "Bairro", "Cidade", 
           "Cep", "Endereço_Completo", "Processo"] 

In [77]:
PATH = r"C:\Users\rsilva\gdrive\anatel\Cassacao\Cassacao_RC\Expedidos"
planilhas = [os.path.splitext(path)[0].replace("_", "/") for path in os.listdir(PATH)]
planilhas.remove('desktop')
planilhas

['53504.006044/2018-76',
 '53504.006045/2018-11',
 '53504.006048/2018-54',
 '53504.006101/2018-17',
 '53504.006105/2018-03',
 '53504.006108/2018-39',
 '53504.006160/2018-95',
 '53504.006238/2018-71',
 '53504.006243/2018-84',
 '53504.006463/2018-16',
 '53504.006778/2018-55',
 '53504.006801/2018-10',
 '53504.006921/2018-17',
 '53504.006932/2018-99',
 '53504.006951/2018-15']

In [119]:
gc = functions.authenticate_google("files/anatel.json")

In [120]:
expedidos = gc.open(title="Expedidos").worksheet(title="Expedidos")
last_row = expedidos.row_count

for num in planilhas:
    sht = gc.open(title=num).get_worksheet(0)
    df = gs.get_as_dataframe(sht, True, dtype=str)
    df.drop(["End_RF", "Situação", "Devedor"], axis=1, inplace=True, errors='ignore')
    df = df.replace('nan', "")
    
    try:
        
        df = df.dropna(subset=["FISTEL"])
        
    except:
        
        df = df.dropna(subset=["Fistel"])
        
        
    df["Processo"] = df.shape[0] * [num]
    gs.set_with_dataframe(expedidos, df, row=last_row + 1, col=1, include_column_header=False)
    last_row += df.shape[0]

In [114]:
num


'53504.006463/2018-16'

In [67]:
for i in range(2, len(sht.get_all_records()) + 2):
    expedidos.append_row(sht.row_values(i) + [numero])
    break

In [5]:
df = gs.get_as_dataframe(sht, dtype=str)

In [10]:
df.drop(columns="CPF", axis=0, inplace=True)

In [3]:
#df = pd.read_excel("files/53504.0069512018-15.xlsx", dtype=str)

In [4]:
#df["Complemento"] = df["Complemento"].str.replace("nan", "")

#df.drop_duplicates(inplace=True)

#df.head()

,Cpf,Nome,Fistel,Validade,Filtro,End_Sede,End_Corresp,Situação,Devedor,Cpf_RF,...,Situação_Cadastral,Fone,Logradouro,Num,Complemento,Bairro,Cidade,UF,Cep,Endereço_Completo
0,16089021881,SAMUEL FELIX DA FONSECA,80106343793,2017-10-19 00:00:00,True,"PASSAGEM VENANCIO DA SILVA, 162\n- JARDIM IMPE...",nan,Entidade devedora (Bloqueada),Sim,160.890.218-81,...,REGULAR,36972282,RUA JOSE TAVARES BASTOS,185,CASA 3,FERRAZOPOLIS,SAO BERNARDO DO CAMPO,SP,09790-510,RUA JOSE TAVARES BASTOS 185 CASA 3 FERRAZOPOLI...
1,255171803,SERGIO LUIZ ZUANAZZI,80106349058,2017-10-19 00:00:00,True,"RUA BARÃO DO RIO BRANCO, 128\n- CENTRO\nCEP: 1...",nan,Entidade devedora (Bloqueada),Sim,002.551.718-03,...,SUSPENSA,721077,RUA BARÃO DO RIO BRANCO,128,,CENTRO,TANABI,SP,15170-000,\nCEP:
2,3592671812,NELSON LUIS FILO,80106375806,2017-10-23 00:00:00,True,"AVENIDA DIOGO JOSE MACHADO, 222\n- JARDIM ITAP...",RUA DIOGO JOSE MACHADO 222 ITAQUERA \nCEP: 082...,Entidade devedora (Bloqueada),Sim,035.926.718-12,...,REGULAR,32156311,RUA DIOGO JOSE MACHADO,222,,ITAQUERA,SAO PAULO,SP,08220-690,RUA DIOGO JOSE MACHADO 222 ITAQUERA \nCEP: 082...
3,31894172884,JOSE RICARDO GUEDES,80106339257,2017-10-24 00:00:00,True,"RUA BENEDITO JOSE GIANNASI, 40\n- ADELINO SIMI...",RUA BENEDITO JOSE GIANNASI 40 ADELINO SIMIONI ...,Atenção: Entidade devedora (Não bloqueada),Sim,318.941.728-84,...,REGULAR,36380175,RUA BENEDITO JOSE GIANNASI,40,,ADELINO SIMIONI,RIBEIRAO PRETO,SP,14071-180,RUA BENEDITO JOSE GIANNASI 40 ADELINO SIMIONI ...
4,29584647415,RIVALDO CHALEGRE DE ATAIDE,80106369750,2017-10-24 00:00:00,True,"RUA CAPITAO PEDRO ESPERIDIAO HOFFER, 238\n- JA...",nan,Atenção: Entidade devedora (Não bloqueada),Sim,295.846.474-15,...,REGULAR,46365676,RUA CAPITAO ESPERIDIAO HOFFER,238,,NOVA POA,POA,SP,08568-700,RUA CAPITAO ESPERIDIAO HOFFER 238 NOVA POA \nC...


In [15]:
df.head()

,Cpf,Nome,Fistel,Validade,Filtro,End_Sede,End_Corresp,Situação,Devedor,Cpf_RF,...,Situação_Cadastral,Fone,Logradouro,Num,Complemento,Bairro,Cidade,UF,Cep,Endereço_Completo
17,29334497904,AILTON DIONISIO SALVADOR,80106345737,2017-11-01 00:00:00,True,"RUA THALES BERNARDES, Nº 130, 130, FIRMA\n- CE...",RUA GALILEU GAIA 000621 PQ VILA MARIA \nCEP: 0...,Atenção: Entidade devedora (Não bloqueada),Sim,293.344.979-04,...,REGULAR,511631,RUA CINCO,206,,CARIJO,CANANEIA,SP,11990-000,RUA CINCO 206 CARIJO \nCEP: 11990-000 CANANEIA/SP
13,53365771891,AIRTON AREDES,80106366300,2017-10-25 00:00:00,True,"RUA EVARISTO DA VEIGA, 103\n- JARDIM INDEPENDE...",RUA ALCINDO SAUL AMARAL 76 JD VISTA ALEGRE \nC...,Atenção: Entidade devedora (Não bloqueada),Sim,533.657.718-91,...,REGULAR,43926975,RUA EVARISTO DA VEIGA,103,CASA,J INDEPENDENCIA,SAO BERNARDO DO CAMPO,SP,09862-420,OUTROS REVARISTO DA VEIGA 103 CASA J INDEPENDE...
9,25863516810,ALEX ROGERIO GERALDO RUBIO,80106380800,2017-10-24 00:00:00,True,"RUA UM, 145, bl 27 apt 22\n- PARQUE RESIDENCIA...",AVENIDA GENERAL ATALIBA LEONEL 3173 AP 195 A P...,Atenção: Entidade devedora (Não bloqueada),Sim,258.635.168-10,...,REGULAR,38381815,RUA JOAO SPANHOL,132,,JD DAS ORQUIDEAS,SUMARE,SP,13174-403,RUA JOAO SPANHOL 132 JD DAS ORQUIDEAS \nCEP: 1...
14,32206943816,DARIL DA SILVA ROCHA,80106352270,2017-10-25 00:00:00,True,"RUA OSVALDO DOS SANTOS, 67, AP 37\n- PARQUE DO...",OUTROS CAETANO ZAFRA CHANCEZ 470 JD JOAO BALLA...,Atenção: Entidade devedora (Não bloqueada),Sim,322.069.438-16,...,REGULAR,62057063,RUA OSVALDO DOS SANTOS,67,APTO 37,PARQUE DOURADO,FERRAZ DE VASCONCELOS,SP,08527-054,RUA OSVALDO DOS SANTOS 67 APTO 37 PARQUE DOURA...
8,8038379845,FLORISVALDO BISPO DOS SANTOS,80106362151,2017-10-24 00:00:00,True,"RUA HUMAITÁ, 106\n- JARDIM EUROPA\nCEP: 194000...",OUTROS DR AMARAL CARVALHO 401 BLOCO 3 APTO 323...,Atenção: Entidade devedora (Não bloqueada),Sim,080.383.798-45,...,REGULAR,2714669,RUA ERNESTO SCALON,26,,JD MORADA SOL,PRESIDENTE VENCESLAU,SP,19400-000,RUA ERNESTO SCALON 26 JD MORADA SOL \nCEP: 194...


In [28]:
usr = "rsilva"
pwd = "Savorthemom3nts"
browser = webdriver.Firefox()

In [29]:
browser = sei.login_sei(browser, usr, pwd)

In [31]:
sei_ = sei.Sei(browser)

sei_.itera_processos()

In [9]:
for i in df.iterrows():
    
    dados = i[1].to_dict()    
    
    tag = sei_.atualizar_contato(dados['Nome'], dados)

In [16]:
p = sei_.go_to_processo(numero)

In [17]:
tipo = "RC_Oficio de Cassação"

In [18]:
def string_endereço(row):
    
    s = 'À<br>' + str(row[1]["Nome"]).title() + '<br>' + str(row[1]["Logradouro"]).title() + ", " + str(row[1]["Num"]) \
         + " " + str(row[1]["Complemento"]).title() + " " + str(row[1]["Bairro"]).title() + "<br>" +  "CEP: " \
         + str(row[1]["Cep"]) +" - " + str(row[1]["Cidade"]).title() + " - " + str(row[1]["UF"]) + "<br><br>" + "<b>FISTEL: " \
         + str(row[1]["Fistel"]) + "</b>"
            
    return s

In [19]:
for row in df.iterrows():    
    
    dados = {'À': string_endereço(row)}    
    
    clip.copy(dados["À"])
    
    p.incluir_oficio(tipo, dados)
    
    sleep(30)

In [30]:
TIPOS = ["Outorga: Rádio do Cidadão", "Outorga: Radioamador", 
         "Outorga: Limitado Móvel Aeronáutico",
         "Outorga: Limitado Móvel Marítimo"]

In [33]:
for p in sei_.get_processos().values():
    
    if p["aviso"] and p["tipo"] in TIPOS:
        
        p = sei_.go_to_processo(p["numero"])
        
        teste = input()
        
        if teste == 'y':
            
            p.edita_postit()
            p.excluir_acomp_especial()
            p.concluir_processo()

In [13]:
for p in sei_.get_processos().values():
    p = sei_.go_to_processo(p["numero"])

    print(p.get_acoes()['Concluir Processo'])
    break

concluirProcesso();


In [21]:
windows = sei_.driver.window_handles

sei_.driver.switch_to_window(windows[0])

sei_.driver.switch_to.default_content()